 --- 
# UCI - Default from Credit Card Clients
---

# Dataset presentation

This dataset contains information on default payments, demographic factors, credit data, history of payment, and bill statements of credit card clients in Taiwan from April 2005 to September 2005.

It can be found here:
https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients

## Variables
There are 25 variables:

* ID: ID of each client
* LIMIT_BAL: Amount of given credit in NT dollars (includes individual and family/supplementary credit
* SEX: Gender (1=male, 2=female)
* EDUCATION: (1=graduate school, 2=university, 3=high school, 4=others, 5=unknown, 6=unknown)
* MARRIAGE: Marital status (1=married, 2=single, 3=others)
* AGE: Age in years
* PAY_0: Repayment status in September, 2005 (-1=pay duly, 1=payment delay for one month, 2=payment delay for two months, ... 8=payment delay for eight months, 9=payment delay for nine months and above)
* PAY_2: Repayment status in August, 2005 (scale same as above)
* PAY_3: Repayment status in July, 2005 (scale same as above)
* PAY_4: Repayment status in June, 2005 (scale same as above)
* PAY_5: Repayment status in May, 2005 (scale same as above)
* PAY_6: Repayment status in April, 2005 (scale same as above)
* BILL_AMT1: Amount of bill statement in September, 2005 (NT dollar)
* BILL_AMT2: Amount of bill statement in August, 2005 (NT dollar)
* BILL_AMT3: Amount of bill statement in July, 2005 (NT dollar)
* BILL_AMT4: Amount of bill statement in June, 2005 (NT dollar)
* BILL_AMT5: Amount of bill statement in May, 2005 (NT dollar)
* BILL_AMT6: Amount of bill statement in April, 2005 (NT dollar)
* PAY_AMT1: Amount of previous payment in September, 2005 (NT dollar)
* PAY_AMT2: Amount of previous payment in August, 2005 (NT dollar)
* PAY_AMT3: Amount of previous payment in July, 2005 (NT dollar)
* PAY_AMT4: Amount of previous payment in June, 2005 (NT dollar)
* PAY_AMT5: Amount of previous payment in May, 2005 (NT dollar)
* PAY_AMT6: Amount of previous payment in April, 2005 (NT dollar)
* default.payment.next.month: Default payment (1=yes, 0=no)


In [4]:
%load_ext autoreload
%autoreload 2

# Useful imports

## Packages

In [242]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [243]:
pd.set_option('display.max_columns', 120)

In [244]:
sns.set_style("darkgrid")

## Data Preparation imports

from sklearn.preprocessing import StandardScaler

## Model imports

In [245]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [246]:
import xgboost as xgb

In [302]:
import lightgbm as lgb

In [248]:
# import catboost

## Metrics imports

In [249]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

In [1104]:
import os
os.environ['PATH'].split(';')

['C:\\Users\\twang\\AppData\\Local\\Continuum\\anaconda3\\envs\\boost',
 'C:\\Users\\twang\\AppData\\Local\\Continuum\\anaconda3\\envs\\boost\\Library\\mingw-w64\\bin',
 'C:\\Users\\twang\\AppData\\Local\\Continuum\\anaconda3\\envs\\boost\\Library\\usr\\bin',
 'C:\\Users\\twang\\AppData\\Local\\Continuum\\anaconda3\\envs\\boost\\Library\\bin',
 'C:\\Users\\twang\\AppData\\Local\\Continuum\\anaconda3\\envs\\boost\\Scripts',
 'C:\\Users\\twang\\AppData\\Local\\Continuum\\anaconda3\\envs\\boost\\bin',
 'C:\\Users\\twang\\AppData\\Local\\Continuum\\anaconda3\\condabin',
 'C:\\Program Files\\Docker\\Docker\\Resources\\bin',
 'C:\\ProgramData\\Oracle\\Java\\javapath',
 'C:\\WINDOWS\\system32',
 'C:\\WINDOWS',
 'C:\\WINDOWS\\System32\\Wbem',
 'C:\\WINDOWS\\System32\\WindowsPowerShell\\v1.0',
 'C:\\Program Files\\PuTTY',
 'C:\\Program Files\\Intel\\WiFi\\bin',
 'C:\\Program Files\\Common Files\\Intel\\WirelessCommon',
 'C:\\Program Files\\Git LFS',
 'C:\\Users\\twang\\AppData\\Local\\Continuum

# Importing our Data easily!

In [250]:
path = 'UCI_Credit_Card.csv'

In [251]:
from dataprep.load import load_raw_data
from dataprep.load import load_data
from dataprep.load import load_data_xy


df = load_data(path)
X_raw, y_raw = load_data_xy(path)

# Protocol for pipelined workflow

Puis on se doit de définir les dictionnaires / classes que l'on va utiliser pour stocker les modèles, leurs scores selon toutes les métriques

Ainsi, on pourra les capitalizer facilement

# Elementary Pipelines

Elementary Pipelines are Pipelines that only do a little processing, such as adding / removing a single feature

# Complex Pipelines

Complex Pipelines are combinations of multiple Elementary Pipelines

# Prediction Pipelines

Prediction Pipelines consists of the following architecture:  
  [Complex Pipeline, Classifier]